In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [45]:
#Calculate Weight for each parameter in the similarity index
def calculate_weight(ref_val, upper_lim, lower_lim, threshold=0.8):
  
  w_lower = math.log(threshold)/math.log(1-((ref_val - lower_lim)/(ref_val + lower_lim)))
  w_upper = math.log(threshold)/math.log(1-((upper_lim - ref_val)/(upper_lim + ref_val)))
  weight = round(math.sqrt(w_lower*w_upper), 2)
  
  return weight

In [46]:
#Calculate Earth Similarity Index for Individual Params

def calc_ESI_param(param, upper_lim, lower_lim, threshold = 0.8):

#   w = {'radius': 0.57, 'density': 1.07, 'escape_velocity': 0.70, 'revolution': 0.70, 'surface_gravity': 0.13, 'surface_temperature': 5.58
#      ,'self': self_defined_weight}
  ref_values = {'radius': 1, 'density': 1, 'escape_velocity': 1, 'revolution': 1, 'surface_gravity': 1, 'surface_temperature': 288}
  
  ESI_P = []

  if param in ref_values:
    ref_val = ref_values[param]
    weight = calculate_weight(ref_val, upper_lim,  lower_lim,  threshold)
  #for i in range(len(param)):
   # V = round(math.pow(1-abs((param[i] - ref_val)/(ref_val + param[i])), weight), 6)
    #ESI_P.append(V)
  
#   return np.array(ESI_P)
    return weight 

In [47]:
#Pass an array of Params to calculate 
#Pass an array of upper lims for respective Params 
#Pass an array of lower lims for respective Params

def calc_ESI(params, upper_lims=None, lower_lims=None):


    #Default Upper Lims
    if upper_lims is None:
        upper_lims = [2]*len(params)
        print(upper_lims)

    #Default Lower Lims 
    if lower_lims is None:
        lower_lims = [0.5]*len(params)
        print(lower_lims)
        
    try:
        #Perform sanity checks 
        len(params) == len(upper_lims) == len(lower_lims)
        
        for i in range(0, len(upper_lims)):
            upper_lims[i]>=lower_lims[i]

        #Calculate Weights    
        weights = []
        for i in range(0, len(params)):
            weight = calc_ESI_param(params[i], upper_lims[i], lower_lims[i])
            weights.append(weight)

        return weights
    
    except ValueError as e:
        print(e)

    


In [48]:
params = ['radius', 'density']
upper_lims=[1.9, 1.5]
lower_lims = [0.5, 0.7]
calc_ESI(params,upper_lims,lower_lims)

[0.57, 1.07]

In [49]:
### MAKE USER ENTER CSV WITH VARIOUS COLUMNS CALCULATE ESI FOR EACH ENTRY 

#1. Read Column Names 
#2. Browse column names and select columns which are in the ESI Formula - Data Cleaning features we need to add 
#3. Give option to add custom parameters - FUTURE WORK 
#4. Calculate weights for each parameters 
#5. How to take upper lims and lower lims - FUTURE WORK 
#6. Calculate ESI 


In [50]:
def readData(csv):

    #1. Read Column Names 
    df = pd.read_csv(csv, index_col=0) 
    cols = list(df.columns)
    print(cols)
    cols_lower = [col.lower() for col in cols]
    print(cols_lower)

    #2. Browse column names and select columns which are in the ESI Formula
    columns = ['radius', 'density', 'escape_velocity', 'revolution']
    
    common_cols = []

    for i in range(0, len(columns)):     
        for j in range(0, len(cols_lower)):
            if columns[i] in cols_lower[j]:
                df.rename({cols[j]: columns[i]}, inplace=True, axis=1)
                common_cols.append(columns[i])

   
    new_df = df[common_cols].copy()   

    return new_df      




new_df = readData(r"D:/Downloads/Rock NESI.csv")

['P. Name Kepler', 'P. Name KOI', 'P. Composition Class', 'P. Mass (EU)', 'P. Radius (EU)', 'P. Density (EU)', 'P. Gravity (EU)', 'P. Esc Vel (EU)', 'P. Teq Mean (K)', 'P. Ts Mean (K)', 'P. Surf Press (EU)', 'P. Period (days)', 'P. Mean Distance (AU)', 'Distance (Km)', 'S. Mass (SU)', 'S.Mass (Kg)', 'Rev (Days)', 'Rev (EU)', 'ESI(R)', 'ESI(D)', 'ESI (Ve)', 'ESI(T)', 'ESI(Rev)', 'ESI(g)', 'ESI_I', 'ESI_S', 'ESI']
['p. name kepler', 'p. name koi', 'p. composition class', 'p. mass (eu)', 'p. radius (eu)', 'p. density (eu)', 'p. gravity (eu)', 'p. esc vel (eu)', 'p. teq mean (k)', 'p. ts mean (k)', 'p. surf press (eu)', 'p. period (days)', 'p. mean distance (au)', 'distance (km)', 's. mass (su)', 's.mass (kg)', 'rev (days)', 'rev (eu)', 'esi(r)', 'esi(d)', 'esi (ve)', 'esi(t)', 'esi(rev)', 'esi(g)', 'esi_i', 'esi_s', 'esi']


In [51]:
#4. Calculate weights for each parameters based on default limits
new_cols = list(new_df.columns)

#OPTIONAL - Pass upper and lower lims or will take default values 
calc_ESI(new_cols,upper_lims, lower_lims)

[0.57, 1.07]

In [52]:
new_cols

['radius', 'density']